# Data scraping and collection pipeline


# Telegram Scraping

### Authenticate with Telegram API

In [7]:
import asyncio
from telethon import TelegramClient

# Replace these with your own API credentials from https://my.telegram.org
api_id = '21423729'
api_hash = '60c830a40a5ff9cb549824b9a6399737'
phone_number = '+251986408483'

# Create the client
client = TelegramClient('session_name', api_id, api_hash)

async def main():
    await client.start(phone_number)
    print("Client Created")
    
    # Your scraping functions here...
    
    await scrape_all_channels(client)
    
    # Optional: Disconnect client when done
    # await client.disconnect()

if __name__ == "__main__":
    asyncio.run(main())

OperationalError: database is locked

### Scrape Data from Telegram Channels

In [5]:
from telethon import functions, types
import pandas as pd

# Assuming 'client' is already defined and started from your previous code
# Make sure 'client' is in the global scope or you pass it to functions properly

# Define scrape_channel function
async def scrape_channel(channel_username, client):
    try:
        channel = await client.get_entity(channel_username)
        messages = await client.get_messages(channel, limit=100)  # Adjust limit as needed
        data = []
        
        for message in messages:
            data.append({
                'date': message.date,
                'text': message.text if message.text else None,
                'media_type': type(message.media).__name__ if message.media else None,
            })
        
        return data
    except Exception as e:
        print(f"Error scraping channel {channel_username}: {e}")
        return []

# List of channels to scrape
channels = ['DoctorsET', 'Chemed', 'lobelia4cosmetics', 'yetenaweg', 'EAHCI']

# Scrape data from all channels
async def scrape_all_channels(client):
    all_data = []
    
    for channel in channels:
        scraped_data = await scrape_channel(channel, client)
        all_data.extend(scraped_data)

    # Convert to DataFrame
    df = pd.DataFrame(all_data)

    # Save to CSV
    df.to_csv('telegram_raw_data.csv', index=False)
    print(f"Scraped data from {len(channels)} channels and saved to 'telegram_raw_data.csv'.")

# Run the scraping function
# Make sure 'client' is accessible here
await scrape_all_channels(client)

NameError: name 'client' is not defined

# Image Scraping for Object Detection


In [9]:
import os

async def download_images(channel_username, save_dir):
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    
    channel = await client.get_entity(channel_username)
    messages = await client.get_messages(channel, limit=50)  # Adjust limit as needed
    
    for i, message in enumerate(messages):
        if message.media:
            await message.download_media(file=os.path.join(save_dir, f'image_{i}.jpg'))

# Example: Download images from a channel
channel_username = 'lobelia4cosmetics'
save_directory = 'downloaded_images'
client.loop.run_until_complete(download_images(channel_username, save_directory))

# Storing Raw Data

### Local Files

In [ ]:
# Save scraped data to a DataFrame
import pandas as pd
df = pd.DataFrame(scraped_data)
df.to_csv('telegram_raw_data.csv', index=False)

### Moving to Local Database

In [ ]:
import sqlite3
import pandas as pd

# Assuming data is already in a DataFrame
df = pd.read_csv('telegram_raw_data.csv')

conn = sqlite3.connect('telegram_data.db')
df.to_sql('raw_messages', conn, if_exists='replace', index=False)
conn.close()

# Monitoring and Logging

### Logging:

In [ ]:
import logging
from datetime import datetime

# Configure logging
logging.basicConfig(
    filename='telegram_scraping_log_{}.log'.format(datetime.now().strftime('%Y%m%d_%H%M%S')),
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)

# Example usage in your scraping script
try:
    # Your scraping code here
    logging.info('Scraping started for channel: %s', channel_name)
    # More logging statements as needed
except Exception as e:
    logging.error('Error occurred while scraping: %s', str(e))
finally:
    logging.info('Scraping process concluded')